In [4]:
# 🚀 Imports
import tensorflow as tf
import numpy as np
import cv2
import gradio as gr
from tensorflow.keras.models import load_model


In [5]:
# ✅ Define custom Dice coefficient and loss
def dice_coefficient(y_true, y_pred, smooth=1):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)


In [6]:
# ✅ Load the model (adjusted path because notebook is inside /notebooks)
model = load_model(
    "../models/lung_segmentation_unet.h5",
    custom_objects={"dice_loss": dice_loss, "dice_coefficient": dice_coefficient},
    compile=False,
    safe_mode=True
)

# Optional: recompile for safety
model.compile(optimizer='adam', loss=dice_loss, metrics=[dice_coefficient])


In [7]:
# 🧠 Set target image size used during training
IMG_SIZE = (256, 256)


In [8]:
# 🩻 Segmentation function for Gradio UI
def segment_lung(image):
    # Convert to grayscale
    image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    image_resized = cv2.resize(image_gray, IMG_SIZE)
    image_array = image_resized.astype("float32") / 255.0
    image_input = np.expand_dims(np.expand_dims(image_array, axis=-1), axis=0)

    # Predict mask
    pred_mask = model.predict(image_input)[0]
    pred_mask = (pred_mask > 0.5).astype(np.uint8) * 255

    # Convert to RGB for overlay
    pred_rgb = cv2.cvtColor(pred_mask, cv2.COLOR_GRAY2RGB)
    original_rgb = cv2.cvtColor(image_resized, cv2.COLOR_GRAY2RGB)

    # Overlay
    overlay = cv2.addWeighted(original_rgb, 0.7, pred_rgb, 0.3, 0)
    return overlay


In [9]:
# 🎯 Launch Gradio App
gr.Interface(
    fn=segment_lung,
    inputs=gr.Image(type="numpy", label="Upload Chest X-ray"),
    outputs=gr.Image(label="Segmented Lungs"),
    title="Lung Segmentation using UNet",
    description="Upload a chest X-ray to segment lungs using a trained UNet model."
).launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step
Created dataset file at: .gradio\flagged\dataset1.csv
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
